In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
py.init_notebook_mode()


In [ ]:
DT=1
optimize_model=False

optimize_model_2=False

Make_submission=True

Make_DT_test=False


max_depth=2
min_child_weight=3.5
n_estimators=1200
learning_rate=0.1



max_depth_2=2
min_child_weight_2=1
n_estimators_2=100
learning_rate_2=0.05


ROOT_DIR="/kaggle/input/covid19-global-forecasting-week-4/"

In [ ]:
train=pd.read_csv(ROOT_DIR+"train.csv")
train.head()

For some countries we also have data for individual regions

In [ ]:
have_states=train[train['Province_State'].notna()].groupby(['Country_Region'], sort=False)['Province_State'].nunique()
print(have_states)

US data prior to March 10 is wrongly set to 0. So update using NY Times

In [ ]:
NY_data=pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
alt_initial=NY_data.groupby(['date','state'])[['cases','deaths']].sum().reset_index(drop=False)
US_data_to_add=alt_initial[alt_initial['date']<'2020-03-10']
US_data_to_add['Country_Region']='US'
US_data_to_add.set_index(['date','state','Country_Region'],drop=True,inplace=True)
train=train.merge(US_data_to_add,left_on=['Date','Province_State','Country_Region'],right_index=True,how='left')
train.loc[train['cases'].notnull(),'ConfirmedCases']=train['cases']
train.loc[train['deaths'].notnull(),'Fatalities']=train['deaths']
train.drop(columns=['cases','deaths'],inplace=True)


In [ ]:
train[(train['Country_Region']=='US') & (train['ConfirmedCases']>0)].groupby(['Province_State'])['Date'].min()

Grouping country names and provinces into variable - location. Adding a true/false variable for Islands.

In [ ]:
def add_location(df_old):
    df=df_old.copy()
    df['Date']=pd.to_datetime(df['Date'])
    df['Country_Region']=df['Country_Region'].fillna('')
    df['Province_State']=df['Province_State'].fillna('')
    df['location']=df['Province_State'].astype('str')+" "+df['Country_Region'].astype('str')
    
    df['Island']=False 
    df.loc[df['Province_State'].str.contains("Islan"),'Island']=True
    df.loc[df['Province_State'].isin(['French Polynesia',
       'Guadeloupe', 'Martinique', 'Mayotte', 'New Caledonia', 'Reunion',
       'Saint Barthelemy','Anguilla', 'Bermuda','Isle of Man', 'Montserrat','Aruba',
       'Curacao','Bonaire, Sint Eustatius and Saba','Saint Pierre and Miquelon','Greenland',
                                     'Hawaii']),'Island']=True 
    
    df.loc[df['Country_Region'].isin(['Diamond Princess', 'MS Zaandam']),'Island']=True 
    
    return df

In [ ]:
train=add_location(train)

In [ ]:
train[(train['Province_State']!="") & (train['Island']==False)]['Province_State'].unique()

Locations with less than 5 cases:

In [ ]:
max_cases_old=train[train['Date']<'2020-04-05'].groupby(['location'], sort=False)['ConfirmedCases'].max()
max_cases=train.groupby(['location'], sort=False)['ConfirmedCases'].max()
print("Now: {}\r\nSeven Days ago: {}".format(len(max_cases[max_cases<5]),len(max_cases_old[max_cases_old<5])))

In [ ]:
max_cases[max_cases<5]

In [ ]:
train.set_index('location',inplace=True)

train['day_of_year']=train['Date'].dt.dayofyear
train['day_of_week']=train['Date'].dt.dayofweek


first_day=train[(train['ConfirmedCases']>0)].groupby(['location'], sort=False)['day_of_year'].min()
first_day.rename('first_day',inplace=True)

day_ten=train[(train['ConfirmedCases']>10)].groupby(['location'], sort=False)['day_of_year'].min()
day_ten.rename('day_ten',inplace=True)

day_hundred=train[(train['ConfirmedCases']>100)].groupby(['location'], sort=False)['day_of_year'].min()
day_hundred.rename('day_hundred',inplace=True)

In [ ]:
def add_days_passed(df_old,first_day,day_ten,day_hundred):
    df=df_old.copy()
    df=pd.concat([df,first_day],axis=1,join='inner')
    
    df['days_passed']=df['day_of_year']-df['first_day']
    df.drop(columns=['first_day'],inplace=True)
    
    #print(first_day) 
    
    
    df.loc[df['days_passed']<0,'days_passed']=-1
    
    df=df.merge(day_ten,left_index=True,right_index=True,how="outer")
    
    df['days_passed_10']=df['day_of_year']-df['day_ten']
    df.loc[df['day_ten'].isna(),'days_passed_10']=-1
    df.loc[df['days_passed_10']<0,'days_passed_10']=-1
    df.drop(columns=['day_ten'],inplace=True)
    
    df=df.merge(day_hundred,left_index=True,right_index=True,how="outer")
    
    df['days_passed_100']=df['day_of_year']-df['day_hundred']
    df.loc[df['day_hundred'].isna(),'days_passed_100']=-1
    df.loc[df['days_passed_100']<0,'days_passed_100']=-1
    df.drop(columns=['day_hundred'],inplace=True)
   
    df['location']=df.index
    
    df.loc[df['location']=='Hubei China','days_passed']+=35
    df.loc[df['location']=='Hubei China','days_passed_10']+=22
    df.loc[df['location']=='Hubei China','days_passed_100']+=4
    
    df.set_index('Id',inplace=True)
    df['Id']=df.index
    return df


In [ ]:
train=add_days_passed(train,first_day,day_ten,day_hundred)

In [ ]:
cases_first_day=train[train['days_passed']==0].groupby(['location'])[['ConfirmedCases']].min()
cases_first_day.rename(columns={'ConfirmedCases':'Cases_first_day'},inplace=True)
#print(cases_first_day)
cases_seven_day=train[train['days_passed']==7].groupby(['location'])[['ConfirmedCases']].min()
cases_seven_day.rename(columns={'ConfirmedCases':'Cases_seven_day'},inplace=True)
#print(cases_six_day)
    
def add_first_days(old_df,cases_first_day,cases_seven_day):
    df=old_df.copy()
    
    df=df.merge(cases_first_day,left_on=['location'],right_index=True,how="left")
    
    df['Cases_first_day']=np.log1p(df['Cases_first_day'])
    df.loc[df['days_passed']<0,'Cases_first_day']=-1
    df.loc[df['location']=='Hubei China','Cases_first_day']=1 
    
    df=df.merge(cases_seven_day,left_on=['location'],right_index=True,how="left")
    
    df['Cases_seven_day']=np.log1p(df['Cases_seven_day'])
    df.loc[df['days_passed']<0,'Cases_seven_day']=-1
    df.loc[df['location']=='Hubei China','Cases_seven_day']=3
    
    return df

In [ ]:
train=add_first_days(train,cases_first_day,cases_seven_day)

train.head()

In [ ]:
country_stat=pd.read_csv('../input/countryinfo/covid19countryinfo.csv')
country_stat = country_stat[country_stat['region'].isnull()] 

us_stat=pd.read_csv('../input/covid19-state-data/COVID19_state.csv')
us_stat.rename(columns={'State':'location','Population':'pop','Pop Density':'density','Smoking Rate':'smokers','Respiratory Deaths':'lung'},inplace=True)
us_stat['location']+=" US"
us_stat.set_index('location',inplace=True)

country_metadata=pd.read_csv("../input/covid19-forecasting-metadata/region_metadata.csv")
country_metadata['location']=country_metadata['Province_State'].astype('str')+" "+country_metadata['Country_Region'].astype('str')
country_metadata.rename(columns={'population':'pop'},inplace=True)
country_metadata.set_index('location',inplace=True)


def add_country_stat(old_df,country_stat,us_stat,country_metadata):
    df=old_df.copy()
    df=df.merge(country_stat[['country','pop','medianage','sex65plus','lung','smokers','density']],left_on=['Country_Region'],right_on=['country'],how='left')
    df.drop(columns=['country'],inplace=True)
    
    df['pop']=df['pop'].fillna(1000)
    df['pop']=df['pop'].apply(lambda x: int(str(x).replace(',', '')))
    
    #df['gdp2019']=df['gdp2019'].fillna(0)
    #df['gdp2019']=df['gdp2019'].apply(lambda x: int(str(x).replace(',', '')))
    #df['gdp2019']=df['gdp2019']/df['pop']
    
    
    df['density']=df['density'].fillna(0)
    df['medianage']=df['medianage'].fillna(0) #highly correlated with Temperature... 
    #df['sexratio']=df['sexratio'].fillna(1)
    df['sex65plus']=df['sex65plus'].fillna(1)
    df['lung']=df['lung'].fillna(24)
    df['smokers']=df['smokers'].fillna(24)
    #df['lung']=df['lung']*df['pop']
    
    df.set_index('location',inplace=True)
    df.update(us_stat[['pop','density','smokers','lung']])
    
    df.update(country_metadata[['pop','density']])
    
    df['pop']=np.log1p(df['pop'])
    
    df['location']=df.index
    df.set_index('Id',inplace=True)
    df['Id']=df.index
    
    
    
    return df
    

train=add_country_stat(train,country_stat,us_stat,country_metadata)

In [ ]:
country_stat.info()

In [ ]:
us_stat.info()

In [ ]:
weather_info=pd.read_csv("../input/climate-change-earth-surface-temperature-data/GlobalLandTemperaturesByCountry.csv")
weather_info['dt']=pd.to_datetime(weather_info['dt'])
weather_info=weather_info[weather_info['dt']>'2000-12-30']
weather_info['month']=weather_info['dt'].dt.month
weather_info.drop(weather_info[weather_info['Country'].isin(
    ['Denmark', 'France', 'Netherlands','United Kingdom'])].index,axis=0,inplace=True)

weather_info.replace(
   ['Denmark (Europe)', 'France (Europe)', 'Netherlands (Europe)', 'United Kingdom (Europe)'],
   ['Denmark', 'France', 'Netherlands', 'United Kingdom'],inplace=True)

weather_info.replace({
    'Antigua And Barbuda':'Antigua and Barbuda',
    'Bosnia And Herzegovina':'Bosnia and Herzegovina',
    'Congo (Democratic Republic Of The)':'Congo (Kinshasa)',
    'Congo':'Congo (Brazzaville)',
    'Palestina':'West Bank and Gaza',
    'Cape Verde':'Cabo Verde',
    "Côte D'Ivoire":"Cote d'Ivoire",
    'Trinidad And Tobago':'Trinidad and Tobago',
    'Saint Kitts And Nevis':'Saint Kitts and Nevis',
    'Czech Republic':'Czechia',
    'Swaziland':'Eswatini',
    'Guinea Bissau':'Guinea-Bissau',
    'South Korea':'Korea, South', 
    'Macedonia':'North Macedonia',
    'Saint Vincent And The Grenadines':'Saint Vincent and the Grenadines',
    'Sao Tome And Principe':'Sao Tome and Principe',
    'Taiwan':'Taiwan*', 
    'Timor Leste':'Timor-Leste',
    'United States':'US'
},inplace=True)
weather_country=weather_info.groupby(['Country','month'])['AverageTemperature'].mean()


state_weather_info=pd.read_csv("../input/climate-change-earth-surface-temperature-data/GlobalLandTemperaturesByState.csv")
state_weather_info.replace({'United States':'US','Georgia (State)':'Georgia','District Of Columbia':'District of Columbia'},inplace=True)
state_weather_info['dt']=pd.to_datetime(state_weather_info['dt'])
state_weather_info=state_weather_info[state_weather_info['dt']>'2000-12-30']
state_weather_info['month']=state_weather_info['dt'].dt.month
weather_state=state_weather_info[state_weather_info['Country'].isin(have_states.index)].groupby(['Country','State','month'])['AverageTemperature'].mean()

In [ ]:
def add_temperature(old_df,weather_country,weather_state):
    df=old_df.copy()
    df['Month']=df['Date'].dt.month
    df=df.merge(weather_country,how="left",left_on=['Country_Region','Month'],right_index=True)
    df=df.merge(weather_state,how="left",left_on=['Country_Region','Province_State','Month'],right_index=True)
    df.loc[df['AverageTemperature_y'].notnull(),'AverageTemperature_x']=df['AverageTemperature_y']
    
    inhirit=[['Holy See','Italy'],['South Sudan','Sudan'],['Kosovo','Serbia'],['Brunei','Malaysia'],['Maldives','Sri Lanka']]
    
    for pairs in inhirit:
        for m in df['Month'].unique():
            df.loc[(df['Country_Region']==pairs[0]) & (df['Month']==m),'AverageTemperature_x']=df.loc[(df['Country_Region']==pairs[0]) & (df['Month']==m)]['AverageTemperature_x'].fillna(weather_country[(pairs[1],m)])
        
    df.drop(columns=['AverageTemperature_y','Month'],inplace=True)
    df.rename(columns={'AverageTemperature_x':'AverageTemperature'},inplace=True)
        
    
    return df

train=add_temperature(train,weather_country,weather_state)
    

In [ ]:
train[train['AverageTemperature'].isnull()]['Country_Region'].unique()

In [ ]:
border_info=pd.read_csv("../input/country-borders/border_info.csv")
#border_info.drop(columns=["country_code","country_border_code"],inplace=True)
border_info.replace({'United States of America':'US',
                    'United Kingdom of Great Britain and Northern Ireland':'United Kingdom',
                    'Bolivia (Plurinational State Of)':'Bolivia',
                    'Brunei Darussalam':'Brunei',
                    'Gambia (the)':'Gambia',
                     'Congo (the Democratic Republic of the)':'Congo (Kinshasa)',
                     'Cote d’Ivoire':"Cote d'Ivoire",
                     "Iran (Islamic Republic of)":'Iran',
                     "Korea (the Republic of)":'Korea, South',
                    "Lao People's Democratic Republic":'Laos',
                     "Moldova (the Republic of)":'Moldova',
                     'Myanmar':'Burma',
                     'Palestine, State of':'West Bank and Gaza',
                     "Russian Federation":'Russia',
                    "Syrian Arab Republic":'Syria',
                     "Taiwan (Province of China)":'Taiwan*',
                    "Tanzania (the United Republic of)":'Tanzania',
                     "Venezuela (Bolivarian Republic of)":'Venezuela',
                     "Viet Nam":'Vietnam'},inplace=True)
border_info=border_info.fillna("")

border_info=pd.concat([border_info,pd.DataFrame({'country_name':["Kosovo","Kosovo","Kosovo","Congo (Brazzaville)","Congo (Brazzaville)","Congo (Brazzaville)","Congo (Brazzaville)","Montenegro","Serbia","North Macedonia","Congo (Kinshasa)","Gabon","Cameroon","Central African Republic"],
                                                'country_border_name':["Montenegro","Serbia","North Macedonia","Congo (Kinshasa)","Gabon","Cameroon","Central African Republic","Kosovo","Kosovo","Kosovo","Congo (Brazzaville)","Congo (Brazzaville)","Congo (Brazzaville)","Congo (Brazzaville)"]})],axis=0)



#border_info.to_csv("border_info.csv")


In [ ]:
#set(border_info['country_name'].unique()).difference(set(train['Country_Region'].unique()))
set(train['Country_Region'].unique()).difference(set(border_info['country_name'].unique()))

In [ ]:
islands=border_info[border_info['country_border_name']=='']['country_name'].unique()
islands=np.delete(islands,np.where(islands=="Australia"))
train.loc[train['Country_Region'].isin(islands),'Island']=True 

In [ ]:
response_info=pd.read_csv("../input/oxford-gouvernement-response-tracker/OxCGRT_Download_150420_152128_Full.csv")
response_info['Date']=pd.to_datetime(response_info['Date'], format='%Y%m%d', errors='ignore')
response_info['Date']=response_info['Date'].dt.dayofyear
travel_control=response_info[response_info['S7_International travel controls']>1].groupby(['CountryName'])['Date'].min()
travel_control=travel_control.rename('Date_travel_restriction')
#travel_control=response_info[response_info['S2_Workplace closing']>1].groupby(['CountryName'])['Date'].min()
travel_control_1=response_info[(response_info['S6_Restrictions on internal movement']>1)].groupby(['CountryName'])['Date'].min()
travel_control_1=travel_control_1.rename('Date_travel_restriction_1')
#travel_control_2=response_info[response_info['S3_Cancel public events']>1].groupby(['CountryName'])['Date'].min()
#travel_control_2=travel_control_2.rename('Date_travel_restriction_2')
travel_control=pd.concat([travel_control,travel_control_1],axis=1,sort=True)


def add_restriction_info(old_df,travel_control):
    df=old_df.copy()
    df=df.merge(travel_control,left_on='Country_Region',right_index=True,how="left")
    df['days_since_restriction']=df['day_of_year']-df['Date_travel_restriction']
    df.loc[df['days_since_restriction']<0,'days_since_restriction']=-1
    df.loc[df['Date_travel_restriction'].isna(),'days_since_restriction']=-1

    df['days_since_restriction_1']=df['day_of_year']-df['Date_travel_restriction_1']
    df.loc[df['days_since_restriction_1']<0,'days_since_restriction_1']=-1
    df.loc[df['Date_travel_restriction_1'].isna(),'days_since_restriction_1']=-1

#train['days_since_restriction_2']=train['day_of_year']-train['Date_travel_restriction_2']
#train.loc[train['days_since_restriction_2']<0,'days_since_restriction_2']=-1
#train.loc[train['Date_travel_restriction_2'].isna(),'days_since_restriction_2']=-1

    df.drop(columns=['Date_travel_restriction','Date_travel_restriction_1'],inplace=True)
    
    return df

train=add_restriction_info(train,travel_control)

In [ ]:
response_info.columns

In [ ]:
from itertools import product as it_product
def expand_grid(data_dict):
  rows = it_product(*data_dict.values())
  return pd.DataFrame.from_records(rows, columns=data_dict.keys())

In [ ]:
skel=expand_grid({'Index':border_info.index,'Date':train['Date'].unique()})

country_info=train.groupby(['Date','Country_Region'])['ConfirmedCases'].sum()

skel=skel.merge(border_info, how='inner', left_on=['Index'],right_index=True)
skel=skel.merge(country_info, how='inner', 
                left_on=['Date','country_border_name'],right_on=['Date','Country_Region'])

In [ ]:
from datetime import timedelta
skel['Date']=skel['Date']+timedelta(days=DT)
border_cases=skel.groupby(['country_name','Date'])['ConfirmedCases'].sum()
len(skel['country_name'].unique())

In [ ]:
train=train.merge(border_cases, how='left', left_on=['Country_Region','Date'],right_on=['country_name','Date'])
train['ConfirmedCases_y']=train['ConfirmedCases_y'].fillna(0)
train.rename(columns={'ConfirmedCases_y':'ConfirmedCases_neighbors','ConfirmedCases_x':'ConfirmedCases'},inplace=True)

In [ ]:
big_train = pd.concat([train,pd.get_dummies(train['location'], prefix='loc')],axis=1)
big_train['ConfirmedCases_neighbors']=np.log1p(big_train['ConfirmedCases_neighbors'])
big_train.reset_index(inplace=True)
big_train.drop(columns=["Id"],inplace=True)

In [ ]:
big_train.shape

In [ ]:
def df_add_deltas(df_old):
    df=df_old.copy()
    df=df.sort_values(by=['location', 'Date'])
    
    df['growth_ConfirmedCases_neighbors']=df.groupby(['location'])['ConfirmedCases_neighbors'].diff()
    
    df['d_ConfirmedCases'] = df.groupby(['location'])['ConfirmedCases'].diff()
    df['d_Fatalities'] = df.groupby(['location'])['Fatalities'].diff()
    
        
    df.loc[df['d_Fatalities']<0,'d_Fatalities']=0
    df.loc[df['d_ConfirmedCases']<0,'d_ConfirmedCases']=0
    
    df['prev_ConfirmedCases']=df['ConfirmedCases']-df['d_ConfirmedCases']
    df['prev_Fatalities']=df['Fatalities']-df['d_Fatalities']
    
      
    df['growth_ConfirmedCases']=df['d_ConfirmedCases']/(df['prev_ConfirmedCases']+1)
    df['growth_Fatalities']=df['d_Fatalities']/(df['prev_Fatalities']+1)
    
    df['growth_ConfirmedCases']=np.log1p(df['growth_ConfirmedCases'])
    df['growth_Fatalities']=np.log1p(df['growth_Fatalities'])
    
    df['prev_ConfirmedCases']=np.log1p(df['prev_ConfirmedCases'])
    df['prev_Fatalities']=np.log1p(df['prev_Fatalities'])
    
    #df.drop(columns=['prev_ConfirmedCases','prev_Fatalities'], inplace=True)
    
    first_day_stat=df[df['Date']=='2020-01-22']
    
    df.drop(df[df['Date']=='2020-01-22'].index, inplace=True)
    
    return df,first_day_stat

big_train,first_day_stat=df_add_deltas(big_train)

In [ ]:
big_train.reset_index(inplace=True,drop=True)

In [ ]:
X=big_train.drop(columns=['Province_State','Country_Region','Date','ConfirmedCases','Fatalities','location',
                          'd_ConfirmedCases','d_Fatalities','growth_ConfirmedCases','growth_Fatalities'])

y=big_train['d_ConfirmedCases']
y_2=big_train['d_Fatalities']

In [ ]:
max_day=X['day_of_year'].max()
mask_train=X['day_of_year']<max_day-DT+1
mask_test=X['day_of_year']>=max_day-DT+1

In [ ]:
X_train=X[mask_train]
X_test=X[mask_test]


y_train=y[mask_train]
y_test=y[mask_test]

y_train_2=y_2[mask_train]
y_test_2=y_2[mask_test]

In [ ]:
X_test['day_of_year'].nunique()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

corr = big_train[['d_ConfirmedCases','d_Fatalities','days_passed','ConfirmedCases_neighbors','pop',
                  'medianage','sex65plus','lung','smokers','density','Island','growth_ConfirmedCases',
                  'growth_Fatalities','AverageTemperature','days_passed_10'#,'prev_ConfirmedCases','prev_Fatalities'
                 ]].corr("spearman")
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(12,12))
    ax = sns.heatmap(corr, annot=True,cmap="YlGnBu",vmax=.3, square=True, linewidths=.3)
plt.show()

In [ ]:
len(big_train[(big_train['growth_ConfirmedCases']==0)]['growth_ConfirmedCases'])

In [ ]:
len(big_train[(big_train['growth_ConfirmedCases']>0)]['growth_ConfirmedCases'])

In [ ]:
big_train[(big_train['growth_ConfirmedCases']>0) & (big_train['growth_ConfirmedCases']<=1)]['growth_ConfirmedCases'].hist(bins=50)

In [ ]:
fig1 = px.scatter(big_train[(big_train['growth_ConfirmedCases']<2) & (big_train['Country_Region']=='Italy')],x='Date',y='growth_ConfirmedCases')
fig1.show()

In [ ]:
fig1 = px.scatter(big_train[(big_train['Country_Region']=='Italy')],x='Date',y='d_ConfirmedCases')
fig1.show()

In [ ]:
fig1 = px.scatter(big_train,x='days_passed',y='d_ConfirmedCases')
fig1.update_layout(yaxis_type="log")

fig1.show()

There is a rather good distinction between countries with a lot of cases in their neighbors and those with only a little.

In [ ]:
fig1 = px.scatter(big_train,x='days_passed_10',y='d_ConfirmedCases',color='ConfirmedCases_neighbors')
fig1.update_layout(yaxis_type="log")

fig1.show()

In [ ]:
fig1 = px.scatter(big_train[big_train['Country_Region']!='China'],x='days_since_restriction',y='d_ConfirmedCases',color='location')
fig1.update_layout(yaxis_type="log")

trend=big_train[big_train['Country_Region']!='China'][big_train['days_since_restriction']<70].groupby('days_since_restriction')['d_ConfirmedCases'].mean()

fig1.add_trace(go.Scatter(
        x=trend.index,
        y=trend,
        mode="lines",
        line=go.scatter.Line(color="red"),
        showlegend=False))

fig1.show()

In [ ]:
fig1 = px.scatter(big_train[big_train['Country_Region']!='China'],x='days_since_restriction_1',y='d_ConfirmedCases',color='location')
fig1.update_layout(yaxis_type="log")

trend=big_train[big_train['Country_Region']!='China'][big_train['days_since_restriction_1']<70].groupby('days_since_restriction_1')['d_ConfirmedCases'].mean()

fig1.add_trace(go.Scatter(
        x=trend.index,
        y=trend,
        mode="lines",
        line=go.scatter.Line(color="red"),
        showlegend=False))

fig1.show()

In [ ]:
X_train.drop(columns=['day_of_year'],inplace=True)  #including day of year makes things worse RMLSE goes up from 0.49 to 0.7
X_test.drop(columns=['day_of_year'],inplace=True)   #including day of year makes things worse RMLSE goes up from 0.49 to 0.7

X_train.drop(columns=['day_of_week'],inplace=True)  #including day of week makes things worse RMLSE goes up from 0.49 to 0.57
X_test.drop(columns=['day_of_week'],inplace=True)   #including day of week makes things worse RMLSE goes up from 0.49 to 0.57

X.drop(columns=['day_of_year'],inplace=True)  
X.drop(columns=['day_of_week'],inplace=True)   



In [ ]:
X.drop(columns=['index'],inplace=True)   
X_train.drop(columns=['index'],inplace=True)
X_test.drop(columns=['index'],inplace=True)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
reg = xgb.XGBRegressor(n_estimators=n_estimators,
    max_depth=max_depth,
    min_child_weight=min_child_weight,
    learning_rate=learning_rate,
    random_state=42)
reg_2 = xgb.XGBRegressor(n_estimators=n_estimators_2,
    max_depth=max_depth_2,
    min_child_weight=min_child_weight_2,
    learning_rate=learning_rate_2,random_state=42)

In [ ]:
if Make_DT_test:
    reg.fit(X_train,np.log1p(y_train))

In [ ]:
if Make_DT_test:
    plot = xgb.plot_importance(reg, max_num_features=25)

In [ ]:
if Make_DT_test:
    y_pred = reg.predict(X_test)
    np.sqrt(mean_squared_error(y_pred,np.log1p(y_test)))

In [ ]:
if Make_DT_test:
    X_train_2=X_train.copy().drop(columns=['medianage','density'])
    X_train_2['d_confirmed']=np.log1p(y_train)  
    X_test_2=X_test.copy().drop(columns=['medianage','density'])
    X_test_2['d_confirmed']=y_pred

In [ ]:
if Make_DT_test:
    reg_2.fit(X_train_2,np.log1p(y_train_2))

In [ ]:
if Make_DT_test:
    plot = xgb.plot_importance(reg_2, max_num_features=15)

In [ ]:
if Make_DT_test:
    y_pred_2 = reg_2.predict(X_test_2)
    np.sqrt(mean_squared_error(y_pred_2,np.log1p(y_test_2)))

In [ ]:
big_train=big_train.drop(columns=["index"])

In [ ]:
test=pd.read_csv(ROOT_DIR+"test.csv")
test.rename(columns={'ForecastId':'Id'},inplace=True)
test=add_location(test)

test.set_index('location',inplace=True)

test['day_of_year']=test['Date'].dt.dayofyear
test['day_of_week']=test['Date'].dt.dayofweek
test=add_days_passed(test,first_day,day_ten,day_hundred)
test=add_first_days(test,cases_first_day,cases_seven_day)
test=add_country_stat(test,country_stat,us_stat,country_metadata)
test=add_temperature(test,weather_country,weather_state)
test.loc[train['Country_Region'].isin(islands),'Island']=True 
test=add_restriction_info(test,travel_control)

test.head()

In [ ]:
test.shape

In [ ]:
def create_prediction(old_big_train,old_test,border_info,verbosity=0):

    big_train=old_big_train.copy()
    test=old_test.copy()
    

    known=big_train['Date'].unique()
    
    days_to_predict=test['Date'].unique()
    days_to_predict.sort()


    results=[]
    full_results=[]

    for d in days_to_predict:
        if verbosity>0: print("Predicting {}".format(d))
        if d in known:
            if verbosity>0: print("Data Known")
        
            X=big_train.drop(columns=['Province_State','Country_Region','ConfirmedCases','Fatalities','location','Date',
                                  'day_of_year','day_of_week','d_ConfirmedCases','d_Fatalities','growth_Fatalities',
                                      'growth_ConfirmedCases'])

            y=big_train['d_ConfirmedCases']
            y_2=big_train['d_Fatalities']
        
            mask_train=big_train['Date']<d
            mask_val=big_train['Date']==d
        
            X_train=X[mask_train]
            y_train=y[mask_train]
            y_train_2=y_2[mask_train]
        
            X_val=X[mask_val]
            y_val=y[mask_val]
            y_val_2=y_2[mask_val]
        
            reg = xgb.XGBRegressor(n_estimators=n_estimators,
                                   max_depth=max_depth,
                                   min_child_weight=min_child_weight,
                                   learning_rate=learning_rate,
                                   #subsample=0.95,
                                   random_state=42)
            reg_2 = xgb.XGBRegressor(n_estimators=n_estimators_2,
                                   max_depth=max_depth_2,
                                   min_child_weight=min_child_weight_2,
                                   learning_rate=learning_rate_2,random_state=42)
        
            reg.fit(X_train,np.log1p(y_train))
        
            y_pred = reg.predict(X_val)
            if verbosity>0: print("MSLE {}".format(mean_squared_error(y_pred,np.log1p(y_val))))
        
            X_train_2=X_train.copy().drop(columns=['medianage','density'])
            X_train_2['d_ConfirmedCases']=np.log1p(y_train)  #0.4412899060661785 <- without , with - 0.4463  
            X_val_2=X_val.copy().drop(columns=['medianage','density'])
            X_val_2['d_ConfirmedCases']=y_pred
        
            reg_2.fit(X_train_2,np.log1p(y_train_2))
        
            y_pred_2 = reg_2.predict(X_val_2)
        
            if verbosity>0: print("MSLE {}".format(mean_squared_error(y_pred_2,np.log1p(y_val_2))))
        
        #result=X_test[['']]
        elif d-np.timedelta64(86400000000000,'ns') in known:
            if verbosity>0: print("Data Known")
        
            X=big_train.drop(columns=['Province_State','Country_Region','ConfirmedCases','Fatalities','location','Date',
                                  'day_of_year','day_of_week','d_ConfirmedCases','d_Fatalities','growth_Fatalities',
                                      'growth_ConfirmedCases'])

            y=big_train['d_ConfirmedCases']
            y_2=big_train['d_Fatalities']
        
            mask_train=big_train['Date']<d
        
            X_train=X[mask_train]
            y_train=y[mask_train]
            y_train_2=y_2[mask_train]
        
        
            reg = xgb.XGBRegressor(n_estimators=n_estimators,
                                   max_depth=max_depth,
                                   min_child_weight=min_child_weight,
                                   learning_rate=learning_rate,
                                   #subsample=0.95,
                                   random_state=42)
            reg_2 = xgb.XGBRegressor(n_estimators=n_estimators_2,
                                   max_depth=max_depth_2,
                                   min_child_weight=min_child_weight_2,
                                   learning_rate=learning_rate_2,
                                     random_state=42)
        
            reg.fit(X_train,np.log1p(y_train))
        
            X_train_2=X_train.copy().drop(columns=['medianage','density'])
            X_train_2['d_ConfirmedCases']=np.log1p(y_train)  #0.4412899060661785 <- without , with - 0.4463  
            
            reg_2.fit(X_train_2,np.log1p(y_train_2))
        
        
        
        X_test=test[test['Date']==d]
    
        day=X_test['day_of_year'].iloc[0]
    
        country_info=big_train[big_train['day_of_year']==day-1].groupby(['Country_Region'])['ConfirmedCases','prev_ConfirmedCases'].sum()
    
        border_cases=border_info.merge(country_info, how='inner', 
                left_on=['country_border_name'],right_on=['Country_Region'])
    
        border_cases=border_cases.groupby(['country_name'])['ConfirmedCases','prev_ConfirmedCases'].sum()
        border_cases=border_cases.rename(columns={'ConfirmedCases':'ConfirmedCases_neighbors','prev_ConfirmedCases':'prev_ConfirmedCases_neighbors'})
    
    
        X_test=X_test.merge(border_cases, how='left', left_on=['Country_Region'],right_on=['country_name'])
        X_test['ConfirmedCases_neighbors']=X_test['ConfirmedCases_neighbors'].fillna(0)
        X_test['prev_ConfirmedCases_neighbors']=X_test['prev_ConfirmedCases_neighbors'].fillna(0)
    
        X_test = pd.concat([X_test,pd.get_dummies(X_test['location'], prefix='loc')],axis=1)

        
        X_test['ConfirmedCases_neighbors']=np.log1p(X_test['ConfirmedCases_neighbors'])
        X_test['prev_ConfirmedCases_neighbors']=np.log1p(X_test['prev_ConfirmedCases_neighbors'])
        
        X_test['growth_ConfirmedCases_neighbors']=X_test['ConfirmedCases_neighbors']-X_test['prev_ConfirmedCases_neighbors']
        
        X_test.drop(columns=['prev_ConfirmedCases_neighbors'],inplace=True)
        
        
        X_test=X_test.merge(big_train[big_train['day_of_year']==day-1][['location','ConfirmedCases','Fatalities']], how='left', 
                 left_on=['location'],right_on=['location'])
        X_test.rename(columns={'ConfirmedCases':'prev_ConfirmedCases','Fatalities':'prev_Fatalities'},inplace=True)
        
        #print(X_test)
        
        X_test['prev_ConfirmedCases']=np.log1p(X_test['prev_ConfirmedCases'])
        X_test['prev_Fatalities']=np.log1p(X_test['prev_Fatalities'])
        
    
        X_test.set_index('Id',inplace=True)
        #X_test.drop(columns=['Id'],inplace=True)
        
        #print(X_test.columns)
    
    #print(X_test.head(5))
    
        y_test=reg.predict(X_test.drop(columns=['Province_State','Country_Region','location','Date','day_of_year','day_of_week']))
    
    #print(y_test)
    
        X_test['d_ConfirmedCases']=y_test
    
        y_test=reg_2.predict(X_test.drop(columns=['Province_State','Country_Region','location','Date',
                                            'day_of_year','day_of_week','medianage','density']))
    
        X_test['d_Fatalities']=y_test
    
    #print(X_test.shape)
    
        X_test['Id']=X_test.index
        
    
        X_test=X_test.merge(big_train[big_train['day_of_year']==day-1][['location','ConfirmedCases','Fatalities']], how='left', 
                 left_on=['location'],right_on=['location'])
    
    #print(X_test.head(5))
    
    #X_test.set_index('Id',inplace=True)
    
    #print(X_test.shape)
    
        
    
        X_test.set_index('Id',inplace=True,drop=True)
    
        #print(X_test.head(5))
        
        X_test['d_ConfirmedCases']=np.expm1(X_test['d_ConfirmedCases'])
        X_test['d_Fatalities']=np.expm1(X_test['d_Fatalities'])
        
        X_test['d_ConfirmedCases'].clip(0,inplace=True)
        X_test['d_Fatalities'].clip(0,inplace=True)
    
        X_test['ConfirmedCases']+=X_test['d_ConfirmedCases']
        X_test['Fatalities']+=X_test['d_Fatalities']
        
        if not d in known: #updates for the days_passed_* 
        
            to_update=X_test.loc[(X_test['ConfirmedCases']>=10) & (X_test['days_passed_10']<0) & (X_test['day_of_year']==day),'location'].to_numpy()

            for loc in to_update:
                test.loc[(test['location']==loc) & (test['day_of_year']>=day),'days_passed_10']=test.loc[(test['location']==loc) & (test['day_of_year']>=day),'day_of_year']-day
                #print(day)
                #print(test.loc[(test['location']==loc) & (test['day_of_year']>=day),['location','days_passed_10']])

            to_update=X_test.loc[(X_test['ConfirmedCases']>=100) & (X_test['days_passed_100']<0) & (X_test['day_of_year']==day),'location'].to_numpy()

            for loc in to_update:
                test.loc[(test['location']==loc) & (test['day_of_year']>=day),'days_passed_100']=test.loc[(test['location']==loc) & (test['day_of_year']>=day),'day_of_year']-day
                #print(day)
                #print(test.loc[(test['location']==loc) & (test['day_of_year']>=day),['location','days_passed_100']])
       
    
    
        results.append(X_test[['ConfirmedCases','Fatalities']])
        full_results.append(X_test)
    
        if not d in known: #Needed to correctly get data on neighbors         
            big_train=pd.concat([big_train,X_test],sort=False)
        
    return results,full_results
    

In [ ]:
def score_prediction(train,border_info,date_end_train,date_end_new):
    
    df_train=train.copy()
    
    loc_to_drop=df_train[(df_train['Date']==date_end_new)& (df_train['ConfirmedCases']==0)]['location'].unique()
    
    df_train.drop(df_train[df_train['location'].isin(loc_to_drop)].index,inplace=True)

    df_train.drop(columns='loc_'+loc_to_drop,inplace=True)
    
    
    df_test=df_train[df_train['Date']>=date_end_train].drop(columns=['d_ConfirmedCases','d_Fatalities','growth_Fatalities',
                                      'growth_ConfirmedCases','prev_ConfirmedCases','prev_Fatalities','ConfirmedCases_neighbors',
                                                                   'growth_ConfirmedCases_neighbors','ConfirmedCases','Fatalities'])
    df_test['Id']=df_test.index

    df_test.set_index('Id',drop=True,inplace=True)
    df_test['Id']=df_test.index

    df_test.drop(columns=list(filter(lambda c: 'loc_' in str(c), df_test.columns)),inplace=True)


    results,_=create_prediction(df_train[df_train['Date']<date_end_train],df_test,border_info)
    
    mask_9=df_train[df_train['Date']>date_end_train].index
    
    return np.sqrt(mean_squared_error(np.log1p(pd.concat(results).sort_index()).loc[(mask_9)],
                               np.log1p(df_train.loc[(mask_9)][['ConfirmedCases','Fatalities']])))
    
    

In [ ]:
if optimize_model_2:
    scores=[]
    params=[]

    for n_estimators_2 in range(25,201,25):
        for learning_rate_2 in np.arange(0.05,0.31,0.05):
            params.append({'n_estimators_2':n_estimators_2,'learning_rate_2':learning_rate_2})
            score=score_prediction(big_train,border_info,'2020-04-01','2020-03-24')
            scores.append(score)
            print('For params {} score {}'.format(params[-1],scores[-1]))

    print(params[scores.index(min(scores))])

In [ ]:
if optimize_model_2:
    n_estimators_2=params[scores.index(min(scores))]['n_estimators_2']
    learning_rate_2=params[scores.index(min(scores))]['learning_rate_2']

In [ ]:
if optimize_model_2:


    scores=[]
    params=[]

    for max_depth_2 in range(1,5,1):
        for min_child_weight_2 in np.arange(0.5,5.1,0.5):
            params.append({'max_depth_2':max_depth_2,'min_child_weight_2':min_child_weight_2})
        
            score=score_prediction(big_train,border_info,'2020-04-01','2020-03-24')
            scores.append(score)
            print('For params {} score {}'.format(params[-1],scores[-1]))

    print(params[scores.index(min(scores))])

In [ ]:
if optimize_model_2:
    max_depth_2=params[scores.index(min(scores))]['max_depth_2']
    min_child_weight_2=params[scores.index(min(scores))]['min_child_weight_2']

In [ ]:
if optimize_model:
    scores=[]
    params=[]

    for n_estimators in range(300,1400,50):
        for learning_rate in np.arange(0.1,0.31,0.2):
            params.append({'n_estimators':n_estimators,'learning_rate':learning_rate})
            score=score_prediction(big_train,border_info,'2020-04-01','2020-03-24')
            scores.append(score)
            print('For params {} score {}'.format(params[-1],scores[-1]))

    print(params[scores.index(min(scores))])

In [ ]:
if optimize_model:
    n_estimators=params[scores.index(min(scores))]['n_estimators']
    learning_rate=params[scores.index(min(scores))]['learning_rate']

In [ ]:
if optimize_model:
    scores=[]
    params=[]

    for max_depth in range(1,7,1):
        for min_child_weight in np.arange(0.5,5.1,0.5):
            params.append({'max_depth':max_depth,'min_child_weight':min_child_weight})
        
            score=score_prediction(big_train,border_info,'2020-04-01','2020-03-24')
            scores.append(score)
            print('For params {} score {}'.format(params[-1],scores[-1]))
        
    print(params[scores.index(min(scores))])

In [ ]:
if optimize_model:
    max_depth=params[scores.index(min(scores))]['max_depth']
    min_child_weight=params[scores.index(min(scores))]['min_child_weight']

In [ ]:
print("Would be Week 3 score {}".format(score_prediction(big_train,border_info,'2020-04-08','2020-04-01')))

In [ ]:
print("Would be Week 2 score {}".format(score_prediction(big_train,border_info,'2020-04-01','2020-03-24')))

In [ ]:
if Make_submission:
    results,full_results=create_prediction(big_train,test,border_info,verbosity=1)
    
    full_res=pd.concat(full_results)

In [ ]:
if Make_submission==True:
    submission=pd.read_csv(ROOT_DIR+"submission.csv")
    submission.drop(columns=['ConfirmedCases','Fatalities'],inplace=True)
    submission=submission.merge(pd.concat(results),left_on=['ForecastId'],right_index=True).clip(lower=0)
    submission.to_csv('submission.csv',index=False)

In [ ]:
if Make_submission==True:
    trained=big_train[(big_train['loc_ Switzerland']==1) & (big_train['Date']<'2020-04-13')][['Date','ConfirmedCases','Fatalities']]
    prediction=full_res[full_res['loc_ Switzerland']==1][['Date','ConfirmedCases','Fatalities']]
    fig1 = px.scatter(trained, x="Date", y="ConfirmedCases")

    fig1.add_trace(go.Scatter(
            x=prediction["Date"],
            y=prediction["ConfirmedCases"],
            mode="lines",
            line=go.scatter.Line(color="red"),
            showlegend=False))

    fig1.show()

In [ ]:
if Make_submission==True:
    trained=big_train[(big_train['loc_ Italy']==1) & (big_train['Date']<'2020-04-13')][['Date','ConfirmedCases','Fatalities']]
    prediction=full_res[full_res['loc_ Italy']==1][['Date','ConfirmedCases','Fatalities']]
    fig1 = px.scatter(trained, x="Date", y="ConfirmedCases")

    fig1.add_trace(go.Scatter(
            x=prediction["Date"],
            y=prediction["ConfirmedCases"],
            mode="lines",
            line=go.scatter.Line(color="red"),
            showlegend=False))

    fig1.show()

In [ ]:
if Make_submission==True:
    trained=big_train[(big_train['loc_ Ukraine']==1) & (big_train['Date']<'2020-04-13')][['Date','ConfirmedCases','Fatalities']]
    prediction=full_res[full_res['loc_ Ukraine']==1][['Date','ConfirmedCases','Fatalities']]
    fig1 = px.scatter(trained, x="Date", y="ConfirmedCases")

    fig1.add_trace(go.Scatter(
            x=prediction["Date"],
            y=prediction["ConfirmedCases"],
            mode="lines",
            line=go.scatter.Line(color="red"),
            showlegend=False))

    fig1.show()

In [ ]:
if Make_submission==True:
    trained=big_train[(big_train['loc_New York US']==1) & (big_train['Date']<'2020-04-13')][['Date','ConfirmedCases','Fatalities']]
    prediction=full_res[full_res['loc_New York US']==1][['Date','ConfirmedCases','Fatalities']]
    fig1 = px.scatter(trained, x="Date", y="ConfirmedCases")

    fig1.add_trace(go.Scatter(
            x=prediction["Date"],
            y=prediction["ConfirmedCases"],
            mode="lines",
            line=go.scatter.Line(color="red"),
            showlegend=False))

    fig1.show()